In [ ]:
!git clone https://github.com/JPortilloHub/miningAgent.git
#Install Anthropic
#%pip install anthropic

In [ ]:
#Import libraries
import anthropic
import csv
import json
import io
import uuid
from datetime import datetime
import os

# Initialize Anthropic client
client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

# Medical Policy Document
MEDICAL_POLICY = """
Comprehensive Insurance Policy Document: 2024 Edition
Section 1: Membership Tiers Overview

1. Simple Membership
- Monthly Fee: $29.99
- Core Coverage: Basic medical and emergency services only.
- Deductible: $1,000 per incident.
- Dental/Vision: Not included.
- Maximum Annual Payout: $50,000.
- Key Exclusions: Pre-existing conditions, preventative care, and specialized surgery.

2. Advanced Membership
- Monthly Fee: $79.99
- Core Coverage: Comprehensive medical, preventative care (annual physicals), and limited prescription drug coverage.
- Deductible: $500 per incident.
- Dental/Vision: Includes annual dental checkup and $150 towards vision care.
- Maximum Annual Payout: $250,000.
- Key Features: Access to telemedicine services 24/7.

3. Premium Membership
- Monthly Fee: $149.99
- Core Coverage: Full comprehensive medical, unlimited prescription drug coverage, and specialized surgery coverage.
- Deductible: $0 (No deductible).
- Dental/Vision: Full coverage for all routine and major dental/vision services.
- Maximum Annual Payout: Unlimited.
- Key Features: Global emergency travel assistance and a dedicated customer service agent.
"""

In [ ]:
#Define csv log
def json_to_csv_log(data_list, output_filename="agent_log.csv"):
    """
    Converts a Python list of dictionaries (JSON object) into a CSV file.

    Args:
        data_list (list): The input list containing dictionary objects.
        output_filename (str): The name of the output CSV file.
    """

    # 1. Validate that the input is actually a list
    if not isinstance(data_list, list):
        print("Error: Input is not a valid list. Please provide a Python list of dictionaries.")
        return

    if not data_list:
        print("The data list is empty. No data to write.")
        return

    # 2. Determine the field names (header) from the keys of the first object.
    try:
        fieldnames = list(data_list[0].keys())
    except AttributeError:
        print("Error: The items in the list do not appear to be dictionaries.")
        return

    # 3. Write the data to the CSV file.
    try:
        # Use 'w' mode for writing, 'newline=""' handles cross-platform newline issues
        with open(output_filename, 'w', newline='', encoding='utf-8') as csvfile:
            # Create a DictWriter instance
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            # Write the header row
            writer.writeheader()

            # Write all data rows
            writer.writerows(data_list)

        print(f"Successfully wrote {len(data_list)} log entries to {output_filename}")

    except IOError:
        print(f"I/O Error: Could not write to file {output_filename}")


#Define action logging
def log_action(action, case_id, tool_used={}, comment={}):
    """Log actions with timestamp and case ID"""
    log_entry = {
        "action": action,
        "timestamp": datetime.now().isoformat(),
        "caseId": case_id,
        "tool_used": tool_used,
        "comment": comment
    }
    logs.append(log_entry)
    print(f"[LOG] {json.dumps(log_entry)}")

#Define patient data loading
def load_patient_data():
    """Load patient data from CSV string"""
    patients = []
    file_path = 'miningAgent/docs/Patients.csv'

    with open(file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        # Use csv.reader for reading row-by-row
      csv_reader = csv.DictReader(csv_file)

    # Iterate over all subsequent rows
      for row in csv_reader:
          patients.append(row)
    return patients

#Define patient search
def find_patient(patient_data, patients):
    """
    Find patient by matching ALL provided attributes simultaneously.
    patient_data can contain: patient_id, name, surname
    All provided attributes must match for verification to succeed.
    """
    provided_attrs = {}

    # Normalize provided data
    if 'patient_id' in patient_data and patient_data['patient_id']:
        provided_attrs['id'] = str(patient_data['patient_id']).strip()
    if 'name' in patient_data and patient_data['name']:
        provided_attrs['name'] = patient_data['name'].strip().lower()
    if 'surname' in patient_data and patient_data['surname']:
        provided_attrs['surname'] = patient_data['surname'].strip().lower()

    # Must have at least one attribute to search
    if not provided_attrs:
        return None

    # Check each patient against ALL provided attributes
    for patient in patients:
        match = True

        # Check ID if provided
        if 'id' in provided_attrs:
            if str(patient['Id']) != provided_attrs['id']:
                match = False
                continue

        # Check name if provided
        if 'name' in provided_attrs:
            if patient['Name'].lower() != provided_attrs['name']:
                match = False
                continue

        # Check surname if provided
        if 'surname' in provided_attrs:
            if patient['Surname'].lower() != provided_attrs['surname']:
                match = False
                continue

        # If all provided attributes match, return the patient
        if match:
            return patient

    return None

#Define agent tools
def create_agent_tools():
    """Define tools for the AI agent"""
    tools = [
        {
            "name": "verify_patient",
            "description": "Verify patient identity by checking their attributes against the patient database. ALL provided attributes must match simultaneously for verification to succeed. Can accept patient_id, name, and/or surname.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "Patient ID number (optional if other attributes provided)"
                    },
                    "name": {
                        "type": "string",
                        "description": "Patient's first name (optional if other attributes provided)"
                    },
                    "surname": {
                        "type": "string",
                        "description": "Patient's last name/surname (optional if other attributes provided)"
                    }
                },
                "required": []
            }
        },
        {
            "name": "check_coverage",
            "description": "Check if a patient's policy tier covers their medical circumstance based on the policy document.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "The verified patient ID"
                    },
                    "tier": {
                        "type": "string",
                        "description": "The patient's policy tier (Simple, Advanced, or Premium)"
                    },
                    "circumstance": {
                        "type": "string",
                        "description": "The medical circumstance or service the patient is asking about"
                    }
                },
                "required": ["patient_id", "tier", "circumstance"]
            }
        },
        {
            "name": "finalize_decision",
            "description": "Finalize the coverage decision after verification and evaluation. Must be called to close the case.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "patient_id": {
                        "type": "string",
                        "description": "The patient's ID"
                    },
                    "decision": {
                        "type": "string",
                        "enum": ["Covered", "Not Covered", "Escalated"],
                        "description": "The final coverage decision"
                    },
                    "reasoning": {
                        "type": "string",
                        "description": "Brief explanation of the decision"
                    }
                },
                "required": ["patient_id", "decision", "reasoning"]
            }
        }
    ]

    mining_activity_name = {
        tools[0].get("name"): "Patient Verification Completed",
        tools[1].get("name"): "Coverage Checked",
        tools[2].get("name"): "Decision Finalized",
    }
    return tools, mining_activity_name

def process_tool_call(tool_name, tool_input, tool_mining_activity_name, patients, case_id):

    if tool_name == "verify_patient":
        patient = find_patient(tool_input, patients)

        # Build description of what was searched
        search_parts = []
        if 'patient_id' in tool_input and tool_input['patient_id']:
            search_parts.append(f"ID: {tool_input['patient_id']}")
        if 'name' in tool_input and tool_input['name']:
            search_parts.append(f"Name: {tool_input['name']}")
        if 'surname' in tool_input and tool_input['surname']:
            search_parts.append(f"Surname: {tool_input['surname']}")

        search_desc = ", ".join(search_parts) if search_parts else "No attributes"

        if patient:
            log_action(f"{tool_mining_activity_name}", case_id, f"Tool used: {tool_name}")
            return {
                "found": True,
                "patient_id": patient['Id'],
                "name": f"{patient['Name']} {patient['Surname']}",
                "tier": patient['Tier'],
                "country": patient['Country']
            }
        else:
            log_action("Patient not found", case_id, f"Tool used: {tool_name}")
            log_action("Case closed", case_id)
            return {
                "found": False,
                "message": "Patient not found in database",
                "searched_attributes": search_desc
            }

    elif tool_name == "check_coverage":
        patient_id = tool_input['patient_id']
        tier = tool_input['tier']
        circumstance = tool_input['circumstance'].lower()

        log_action(f"{tool_mining_activity_name}", case_id, f"Tool used: {tool_name}")

        # Coverage logic based on policy document
        coverage_rules = {
            "Simple": {
                "covered": ["basic medical", "emergency services", "emergency"],
                "not_covered": ["pre-existing", "preventative", "specialized surgery", "dental", "vision", "prescription"]
            },
            "Advanced": {
                "covered": ["medical", "preventative", "annual physical", "prescription", "dental checkup", "vision", "telemedicine"],
                "not_covered": ["specialized surgery", "major dental"]
            },
            "Premium": {
                "covered": ["medical", "prescription", "specialized surgery", "dental", "vision", "preventative", "emergency", "travel"],
                "not_covered": []
            }
        }

        rules = coverage_rules.get(tier, {})

        # Check if covered
        is_covered = None
        for keyword in rules.get("covered", []):
            if keyword in circumstance:
                is_covered = True
                break

        # Check if explicitly not covered
        for keyword in rules.get("not_covered", []):
            if keyword in circumstance:
                is_covered = False
                break

        if is_covered is None:
            return {
                "result": "unclear",
                "message": "Not enough information to determine coverage"
            }
        elif is_covered:
            return {
                "result": "covered",
                "tier": tier,
                "message": f"{tier} tier covers this service"
            }
        else:
            return {
                "result": "not_covered",
                "tier": tier,
                "message": f"{tier} tier does not cover this service"
            }

    elif tool_name == "finalize_decision":
        patient_id = tool_input['patient_id']
        decision = tool_input['decision']
        reasoning = tool_input['reasoning']

        log_action(f"{tool_mining_activity_name}", case_id, f"Tool used: {tool_name}", f"decision: {decision}")
        log_action("Case closed", case_id)

        return {
            "status": "closed",
            "decision": decision,
            "reasoning": reasoning
        }

    return {"error": "Unknown tool"}

def run_coverage_agent(user_message):
    """Run the agentic AI coverage verification system"""
    # Generate unique case ID for this workflow execution
    case_id = str(uuid.uuid4())

    log_action("Coverage inquiry initiated", case_id)
    log_action("Input message set", case_id)

    patients = load_patient_data()
    tools, mining_activity_name = create_agent_tools()

    system_prompt = f"""You are an insurance coverage verification agent. Your job is to:
1. Ask patients to identify themselves if they haven't provided their ID, name, or surname
2. Verify patient identity using the verify_patient tool
3. If patient not found, ask them to re-enter their information
4. Once verified, evaluate their coverage question based on their policy tier
5. Use the check_coverage tool to determine if they're covered
6. Provide a final decision: "Covered", "Not Covered", or "Escalated" (if unclear)
7. Always finalize with the finalize_decision tool to close the case

Medical Policy Information:
{MEDICAL_POLICY}

Be helpful, professional, and concise. Always use tools to verify information and make decisions."""

    messages = [{"role": "user", "content": user_message}]

    print(f"\n{'='*60}")
    print(f"CASE ID: {case_id}")
    print(f"USER: {user_message}")
    print(f"{'='*60}\n")

    while True:
        response = client.messages.create(
            model="claude-sonnet-4-5-20250929",
            max_tokens=4096,
            tools=tools,
            system=system_prompt,
            messages=messages
        )

        # Process response
        assistant_message = {"role": "assistant", "content": response.content}
        messages.append(assistant_message)

        # Check if there's text content to display
        for block in response.content:
            if block.type == "text":
                print(f"AGENT: {block.text}\n")

        # Check if we should stop
        if response.stop_reason == "end_turn":
            break

        # Process tool uses
        if response.stop_reason == "tool_use":
            tool_results = []

            for block in response.content:
                if block.type == "tool_use":
                    tool_name = block.name
                    tool_input = block.input
                    tool_mining_activity_name = mining_activity_name.get(block.name)

                    print(f"[TOOL CALL] {tool_name}: {json.dumps(tool_input, indent=2)}\n")

                    # Execute tool
                    result = process_tool_call(tool_name, tool_input, tool_mining_activity_name, patients, case_id)

                    print(f"[TOOL RESULT] {json.dumps(result, indent=2)}\n")

                    tool_results.append({
                        "type": "tool_result",
                        "tool_use_id": block.id,
                        "content": json.dumps(result)
                    })

            # Add tool results to messages
            messages.append({"role": "user", "content": tool_results})
        else:
            break

    return logs


In [ ]:
# Log storage
logs = []
all_prompts = []

# Path to file with all prompts
file_path = 'miningAgent/docs/Prompts.csv'

# Read prompts
with open(file_path, mode='r', newline='', encoding='utf-8') as csv_file:
    # Use csv.DictReader to map the header row to keys
    csv_reader = csv.DictReader(csv_file)
    # Iterate over all subsequent rows
    for row in csv_reader:
        # Extract only the value associated with the 'Prompt List' column
        all_prompts.append(row['Prompt List'])

# Example usage
if __name__ == "__main__":
    print("Insurance Coverage AI Agent System")
    print("="*60)

    all_logs = []

    for prompt in all_prompts:
        logs = run_coverage_agent(prompt)
        all_logs.extend(logs)
        logs = []


    json_to_csv_log(all_logs)